In [ ]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install tenacity

In [ ]:
import openai
from openai import OpenAI

# Set up the OpenAI API key
openai.api_key = '<api-key>' #use your open-ai key

client = OpenAI(
    api_key=openai.api_key
)

In [ ]:
import pandas as pd
import json

dataset = pd.read_csv("data_full_v1.csv")
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)
dataset = dataset.sample(frac=1)
dataset = json.loads(dataset.to_json(orient ='records'))

In [ ]:
# Format the COPA query for the GPT-3.5 Turbo model
def format_query(item):
  return f"{item['prompt']}. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit"

# format_query(dataset[0])

In [ ]:
# Send the query to the GPT-3.5 Turbo model and get the response
def query_gpt3_5_turbo(query):
    print(query)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant capable of causal reasoning."},
            {"role": "user", "content": query}
        ]
    )
    # print(response)
    #return response['choices'][0]['message']['content']
    return response.choices[0].message.content

# response = query_gpt3_5_turbo(format_query(dataset[0]))

In [ ]:
import openai
import json
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential, RetryError


correct_count = 0
processed_items = 0
failed_items = []
y_true = []
y_pred = []
processed = []

# Evaluate the accuracy of GPT-3.5 Turbo on the COPA dataset
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(20))
def evaluate_accuracy(item):
      global correct_count
      query = format_query(item)
      # print(query)
      response = query_gpt3_5_turbo(query)
      print(response)
      print(item['ground_truth'])
      correct_hypothesis = ((int(response) == item['ground_truth'])==True)
      # print(correct_hypothesis)
      y_true.append(item['ground_truth'])
      y_pred.append(int(response))
      correct_count += correct_hypothesis
      print("correct_count is ", correct_count)
      print("completed\n")


# Evaluate accuracy
def evaluate_accuracy_wrapper(data):
      global correct_count
      global failed_items
      global processed_items
      for item in data:
          try:
              evaluate_accuracy(item)
              processed_items = processed_items +1
              processed.append(item)
          except RetryError as e:
              print("failed")
              failed_items.append(item)
          if processed_items==1000:
            break
          if processed_items%10==0:
            print('processed_items ', processed_items)
      print("correct_count in wrapper is ", correct_count)
      return correct_count / len(data)

accuracy = evaluate_accuracy_wrapper(dataset)
print(f"Accuracy: {accuracy * 100:.2f}%")


Streaming output truncated to the last 5000 lines.
0
correct_count is  206
completed

Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Encouragement level has a direct effect on studying habit and exam score. Studying habit has a direct effect on exam score.
For students who are not encouraged and do not study hard, the probability of high exam score is 94%. For students who are not encouraged and study hard, the probability of high exam score is 74%. For students who are encouraged and do not study hard, the probability of high exam score is 65%. For students who are encouraged and study hard, the probability of high exam score is 40%. For students who are not encouraged, the probability of studying hard is 59%. For students who are encouraged, the probability of studying hard is 2%.
If we disregard the mediation effect through studying habit, would encouragement level negatively affect exam s

In [ ]:
with open("y_true.txt", "w") as f:
    for s in y_true:
        f.write(str(s) +"\n")

In [ ]:
with open("y_pred.txt", "w") as f:
    for s in y_pred:
        f.write(str(s) +"\n")

In [ ]:
with open("processed.txt", "w") as f:
    for s in processed:
        f.write(str(s) +"\n")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.48      0.49       490
           1       0.53      0.55      0.54       510

    accuracy                           0.52      1000
   macro avg       0.52      0.52      0.52      1000
weighted avg       0.52      0.52      0.52      1000



In [ ]:
response = query_gpt3_5_turbo("""Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Season has a direct effect on sprinkler and weather. Weather has a direct effect on ground. Sprinkler has a direct effect on ground.
The overall probability of rainy season is 13%. For people in the dry season, the probability of wet ground is 30%. For in the rainy season, the probability of wet ground is 40%.
Is wet ground less likely than dry ground overall?. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Season has a direct effect on sprinkler and weather. Weather has a direct effect on ground. Sprinkler has a direct effect on ground.
The overall probability of rainy season is 13%. For people in the dry season, the probability of wet ground is 30%. For in the rainy season, the probability of wet ground is 40%.
Is wet ground less likely than dry ground overall?. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit
""")

Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Season has a direct effect on sprinkler and weather. Weather has a direct effect on ground. Sprinkler has a direct effect on ground.
The overall probability of rainy season is 13%. For people in the dry season, the probability of wet ground is 30%. For in the rainy season, the probability of wet ground is 40%.
Is wet ground less likely than dry ground overall?. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Season has a direct effect on sprinkler and weather. Weather has a direct effect on ground. Sprinkler has a direct effect on ground.
The overall probability of rainy season is 13%. For people in the dry seaso